# Tasks remaining 
1. Read survey papers and 2/3 research papers carefully to know methods and architectures that have worked for this task.

2. Implement a baseline supervised RGB to Depth regression model using a known architecture(such as an encoder-decoder architecture/U-net). Study the filters and diagnose the model.

3. Go through the codebases of proper pytorch research code, and see structure

4. Implement research papers, diagnose models, compare results, including unsupervised/semi-supervised models.

5. Report

# Models trying out

1. [FRCN, Laina et. al.](https://arxiv.org/pdf/1606.00373.pdf) (TF Model available) -- implement Pytorch model from scratch, refer only pytorch/vision repo
2. https://github.com/karoly-hars/DE_resnet_unet_hyb - read the paper, see the approaches used
3. 

In [2]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# In case of OSError: [Errno 107] Transport endpoint is not connected, restart Kernel

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = 'Depth_Estimation'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [3]:
# this changes the working directory to the folder, and allows us to perform version control operations
%cd drive/My\ Drive/$FOLDERNAME

!git config --global user.email "arjun2000ashok@gmail.com"
!git config --global user.name "Arjun Ashok"

/content/drive/My Drive/Depth_Estimation


In [51]:
%ls

 data/                     experiments/   README.md   utils.py
'Depth Estimation.ipynb'   model/         test.py


In [ ]:
# !git rm -r --cached .
!git add .
!git commit -m 'Created mat loader'
!git log

In [57]:
from zipfile import ZipFile

def extract_zip(input_zip):
    input_zip=ZipFile(input_zip)
    return {name: input_zip.read(name) for name in input_zip.namelist()}


nyu_data_zipfile = 'data/raw/nyu_data.zip'
data = extract_zip(nyu_data_zipfile)

nyu2_train = list((row.split(',') for row in (data['data/nyu2_train.csv']).decode("utf-8").split('\n') if len(row) > 0))
nyu2_test = list((row.split(',') for row in (data['data/nyu2_test.csv']).decode("utf-8").split('\n') if len(row) > 0))

In [60]:
import skimage.io as io
import numpy as np
import h5py

# data path
path_to_depth = 'data/labelled/nyu_depth_v2_labeled.mat'

# read mat file
f = h5py.File(path_to_depth, 'r')

# read 0-th image. original format is [3 x 640 x 480], uint8
images_mat = f['images']
depths_mat = f['depths']

N = images_mat.shape[0]

(3, 640, 480)

In [ ]:
# # reshape
# images = np.empty([N, 480, 640, 3])
# images.shape
# img = images_mat[0]
# images_mat[:,0,:,:].T.shape
# images[:,:,:,0] = img[:,0,:,:].T
# images[:,:,:,1] = img[:,1,:,:].T
# images[:,:,:,2] = img[:,2,:,:].T

# # imshow
# img__ = img_.astype('float32')
# io.imshow(img__/255.0)
# io.show()


# # read corresponding depth (aligned to the image, in-painted) of size [640 x 480], float64
# depth = f['depths'][0]

# # reshape for imshow
# depth_ = np.empty([480, 640, 3])
# depth_[:,:,0] = depth[:,:].T
# depth_[:,:,1] = depth[:,:].T
# depth_[:,:,2] = depth[:,:].T

# io.imshow(depth_/4.0)
# io.show()

In [24]:
!python test.py

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 2680, in fromarray
    mode, rawmode = _fromarray_typemap[typekey]
KeyError: ((1, 1, 1), '|u1')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "test.py", line 17, in <module>
    img, depth = train_dset[0]
  File "/content/drive/My Drive/Depth_Estimation/model/dataloader.py", line 52, in __getitem__
    depth = Image.fromarray(depth)
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 2682, in fromarray
    raise TypeError("Cannot handle this data type: %s, %s" % typekey)
TypeError: Cannot handle this data type: (1, 1, 1), |u1
